Welcome to BiksGPT. This is a simple notebook demonstrating how I build a simple Generative Pre-trained Transformer (GPT), from first principles.

This was inspired by Andrej Karpathy's nanoGPT and makemore series.

I shall update and tweak this model as a way to experiment with stuff on my own. Apologies in advance if you find something broken. :)

In [1]:
# We will begin by download the tiny shakespeare dataset, which consists of all of his works in a single txt file
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-02 01:45:39--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-03-02 01:45:39 (17.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
print("Size of the dataset (num of characters):", len(text))

Size of the dataset (num of characters): 1115394


In [4]:
#Exploring the vocabulary of our dataset to see the different characters present in it
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'Size if vocab is: {vocab_size}\n and the characters are: {"".join(chars)}')

Size if vocab is: 65
 and the characters are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
#Building our simple Tokenizer that simply converts our vocab to a unique integer
ch_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_ch = {i:ch for i,ch in enumerate(chars)}

encoder = lambda str: [ch_to_int[char] for char in str] #encodes each character in a string with it's unique int val
decoder = lambda int: ''.join(int_to_ch[i] for i in int) #decodes the characters in their integer forms

In [7]:
#testing out our tokenizer
print(encoder('penguins are the best'))
print(decoder(encoder('penguins are the best')))

[54, 43, 52, 45, 59, 47, 52, 57, 1, 39, 56, 43, 1, 58, 46, 43, 1, 40, 43, 57, 58]
penguins are the best


In [9]:
#applying our tokenizer to our dataset using pytorch

import torch
data = torch.tensor(encoder(text), dtype=torch.long) #encode and convert to torch type tensor
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [11]:
#let's look at a small part of our dataset, before and after encoding
print('Before:')
print(text[:1000])

Before:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [12]:
print('After:')
print(data[:1000])

After:
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  

In [13]:
# getting our dataset ready for training

#first we split our dataset into train and test sets.
# 80% will be kept for train and the rest for test

n = int(0.8*len(data))
train_data = data[:n]
test_data = data[n:]



In [17]:
#chunking or splitting up the train set into chunks/blocks of size 9

# we chunk them up so that the model can learn the importance of 9 characters and their relationship with each other, such that given the first char it will be able to predict the next token (char).
#this is the max context size/length for predictions
chunk_size = 9

#example
print(train_data[:chunk_size+1])
print(decoder((train_data[:chunk_size+1]).numpy()))

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
First Citi


In [18]:
#let's demonstrate how the above would work

x = train_data[:chunk_size]
y = train_data[1:chunk_size+1]

for t in range(chunk_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47


In [20]:
#let's split our training data into batches for efficient parallelism

torch.manual_seed(22)
batch_size = 4 #split our training to 4 parallel batches
chunk_size = 9

def fetch_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - chunk_size, (batch_size,)) #returns batch_size number of random integers between 0 and len(data) - chunksize
  #These random integers will be used as starting indices for selecting chunks of data.
  #now that we have a batch of random parts of our data, we classify them into x (dependent vars) and y (target)
  x = torch.stack([data[i:i+chunk_size] for i in ix])
  #here x is a stack or multiple rows (batch size num of rows) to act as input data
  y = torch.stack([data[i+1:i+chunk_size+1] for i in ix])
  #here y is a stack or multiple rows to act as target/output data for each tensor rows in input

  return x,y

In [22]:
xb, yb = fetch_batch('train')
print('The inputs are:')
print(xb.shape)
print(xb)

print('The ouputs are:')
print(yb.shape)
print(yb)

#As you can see below the input and outputs will be of shape [batch_size, chunk_size]

The inputs are:
torch.Size([4, 9])
tensor([[43, 56,  1, 50, 43, 39, 56, 52,  1],
        [47, 51, 11,  0, 33, 52, 50, 43, 57],
        [57,  1, 47, 58,  1, 43,  5, 43, 52],
        [ 1, 46, 53, 58,  1, 39,  1, 22, 39]])
The ouputs are:
torch.Size([4, 9])
tensor([[56,  1, 50, 43, 39, 56, 52,  1, 57],
        [51, 11,  0, 33, 52, 50, 43, 57, 57],
        [ 1, 47, 58,  1, 43,  5, 43, 52,  1],
        [46, 53, 58,  1, 39,  1, 22, 39, 41]])


In [23]:
#Let's demonstrate a similar prediction example like before:

for b in range(batch_size):
  for t in range(chunk_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [43] the target: 56
when input is [43, 56] the target: 1
when input is [43, 56, 1] the target: 50
when input is [43, 56, 1, 50] the target: 43
when input is [43, 56, 1, 50, 43] the target: 39
when input is [43, 56, 1, 50, 43, 39] the target: 56
when input is [43, 56, 1, 50, 43, 39, 56] the target: 52
when input is [43, 56, 1, 50, 43, 39, 56, 52] the target: 1
when input is [43, 56, 1, 50, 43, 39, 56, 52, 1] the target: 57
when input is [47] the target: 51
when input is [47, 51] the target: 11
when input is [47, 51, 11] the target: 0
when input is [47, 51, 11, 0] the target: 33
when input is [47, 51, 11, 0, 33] the target: 52
when input is [47, 51, 11, 0, 33, 52] the target: 50
when input is [47, 51, 11, 0, 33, 52, 50] the target: 43
when input is [47, 51, 11, 0, 33, 52, 50, 43] the target: 57
when input is [47, 51, 11, 0, 33, 52, 50, 43, 57] the target: 57
when input is [57] the target: 1
when input is [57, 1] the target: 47
when input is [57, 1, 47] the target: 58
when i

In [24]:
#we mainly capture x(input) for the model to train on and the target y (output) would be used to calculate the loss

In [ ]:
#now let's get to the juict stuff and build our NN architecture